In [1]:
import h5py
import numpy as np
import os
import sys
import sklearn.decomposition

sys.path.append("../src")

import localmodule

composers = localmodule.get_composers()
data_dir = localmodule.get_data_dir()
dataset_name = localmodule.get_dataset_name()
eigenprogression_name = "_".join([
    dataset_name,
    "eigenprogression-transforms"])
eigenprogression_dir = os.path.join(
    data_dir, eigenprogression_name)
S2s = []

for composer_str in composers:
    composer_dir = os.path.join(
        eigenprogression_dir, composer_str)
    eigenprogression_names = os.listdir(composer_dir)
    n_pieces = len(eigenprogression_names)

    for piece_id in range(n_pieces):
        eigenprogression_name = eigenprogression_names[piece_id]
        eigenprogression_path = os.path.join(
            composer_dir, eigenprogression_name)

        h5py_file = h5py.File(eigenprogression_path)
        h5py_keys = list(h5py_file.keys())
        h5py_key = h5py_keys[0]
        S2 = h5py_file[h5py_key][:]
        S2s.append(S2)
        h5py_file.close()
        
S2 = np.stack(S2s)
        
def sparsity_ratios(X):
    X = np.reshape(X, (X.shape[0], -1))
    l1_norms = np.linalg.norm(X, 1, axis=1)
    l2_norms = np.linalg.norm(X, 2, axis=1)
    return l2_norms / l1_norms
        
Xs = []
Xs.append(S2[:, 0, 0, 0, 0])
Xs.append(S2[:, :, 0, 0, 0])
Xs.append(S2[:, :, :, 0, 0, 0])
Xs.append(S2[:, :, :, :, 0, 0])
Xs.append(S2[:, :, :, :, :, 0])
Xs.append(S2)
Xs = list(map(lambda X: np.reshape(X, (X.shape[0], -1)), Xs))

mean_sparsity_ratios = list(map(np.mean, map(sparsity_ratios, Xs)))

In [12]:
for X in Xs:
    l1_norm = np.linalg.norm(X, 1, axis=1)
    l2_norm = np.linalg.norm(X.T, 2)
    print(l1_norm / l2_norm)

[ 0.27893108  0.22608681  0.29462078  0.30466232  0.23828371  0.28859892
  0.17744945  0.27596748  0.13360111  0.07891177  0.08337025  0.23323697
  0.17484155  0.12329992  0.09914587  0.1312366   0.23060147  0.12310506
  0.08681087  0.08746987  0.27448171  0.20470989  0.06026139  0.30665883
  0.08635224  0.26047069  0.31701514  0.12579861  0.11536448  0.15303662
  0.16681531  0.30240488  0.29547     0.10441806  0.28777093  0.29430673
  0.15285079  0.1707207   0.29327878  0.06911904  0.16939662  0.11579137
  0.12180606  0.15132073  0.11387198  0.17634118  0.22286333  0.30163151
  0.09185021  0.09213806  0.10040955  0.14687145  0.19878207  0.13410582
  0.10613322  0.09682698  0.30104637  0.14564921  0.17219591  0.25866506
  0.09017752  0.30438179  0.30572701  0.15016073  0.09571324  0.09635775
  0.12818334  0.1417954   0.17660779  0.23884547  0.23504177  0.12652771
  0.1398191   0.1116858   0.20480998  0.12488013  0.07395928  0.15920992
  0.21186799  0.05391866  0.30523852  0.19972068  0

In [ ]:
explained_variance_ratios = []

for X in Xs:
    pca = sklearn.decomposition.PCA()
    pca.fit(X)
    explained_variance_ratios.append(pca.explained_variance_ratio_)

explained_variance_ratios = np.array(explained_variance_ratios)
    
%matplotlib inline
from matplotlib import pyplot as plt

plt.plot(np.log10(explained_variance_ratios));

In [73]:
print(np.mean(sparsity_ratios(S2_wavelets)))
print(np.mean(sparsity_ratios(S2_eigentriads)))
print(np.mean(sparsity_ratios(S2_scattering)))
print(np.mean(sparsity_ratios(S2_eigenprogressions)))
print(np.mean(sparsity_ratios(S2_spiral)))

2.5667
4.59782
6.1332
16.9251
42.4312
